<a id='titl'></a>
Plotly Tutorial / Reference
I have written this tutorial to introduce the team to the plotly visualization software package. Plotly is a powerful tool for developing interactive data visualizations. In addition to being powerful, it is also extremely flexible - perhaps even more than matplotlib. The plotly API is simple, with a shallow learning curve. With it's power and flexibility, one might think that plotly is sub-optimal for rapidly creating "throw-away" plots during exploratory work, and that another package should be used. This is not true - developing publication-ready visualizations with plotly is easy and quick.

I have put together this tutorial / reference using my knowledge of plotly. I am sure it is very incomplete and that there topics I've missed. It is, at least, a start to practical use of plotly.

<a href=#quiz>Practice</a>

In [ ]:
''' some basic imports '''
import math
import datetime as dat
import numpy as np
import pandas as pd
import scipy.stats as stt
from sklearn.linear_model import LinearRegression

In [ ]:
''' generate some data '''

# make some data - X
np.random.seed(42)
p = 5
n = 100
X = np.random.normal(loc=0, scale=1, size=(n,p))
b = [3, 5, 7, -10, -4]
features = ['X%d'%i for i in range(p)]
y = 42 + np.sum(b*X, axis=1) + np.random.normal(loc=0, scale=1, size=(n,))
data = pd.DataFrame(data=X, columns=features)
data['target'] = y

# try to recover the linear relationship
linreg = LinearRegression()
linreg.fit(X=X, y=y)
data['yhat'] = linreg.predict(X=X)
data['resid'] = data.target - data.yhat
data['absresid'] = data['resid'].abs()

# randomly select some observations for train/test, supposedly
data['Test'] = np.random.binomial(n=1, p=0.3, size=(n,1))
data['Train_test'] = np.where(data['Test']==1, 'Test', 'Train')

# add a date column
years = np.random.randint(low=2015, high=2020, size=(n,))
months = np.random.randint(low=1, high=13, size=(n,))
data['a_date'] = [dat.date(y, m, 1) for y, m in zip(years, months)]
data['a_date'] = data['a_date'].astype(np.datetime64)

# add a string column
data['a_string'] = pd.Series(np.random.randint(low=1, high=4, size=(n,))).replace({1:'A', 2:'B', 3:'C'})

display(data.head())

In [ ]:
# save plots to external files as well as in notebook
saveFigs = input('Save figures to external files as well as in the notebook (enter True or 1)?') in ['True', '1']

## What is plotly?
- a plotting package supporting Python, Java, and R
- generates interactive plots in html & javascript
- object oriented, making heavy use of dictionaries for defining plots
- documention is not as good as matplotlib, though has improved since I started using

### click for the [Plotly Python API](https://plotly.com/python/)

## Plotly Installation
- anaconda installation: `conda install -c plotly plotly chart-studio`
- there are some java package dependencies, which conda should install seamlessly
- if using the `pybakken` environment, you already have plotly installed

## Jupyter Lab
- if using jupyter lab, an extension must be installed also: `jupyter labextension install jupyterlab-plotly`

# Mode
- plotly works in two modes - *online* or *offline*

## Online
- in online mode, an account with [the plotly chart studio website](https://chart-studio.plotly.com/) is needed
- the plots generated are saved in your plotly account, along with the source data
- data, code, and results can be either public or private
- with online mode, your code must set the plotly username and API key with `chart_studio.tools.set_credentials_file(username='???', api_key='???')`
- with a plotly free account, there are limits to the number of plots which can be kept

## Offline
- offline mode completely bypasses the plotly website, and keeps all your data on your computer

I use offline mode exclusively.

## Plotly Imports
To use most of the functionality of plotly - at least however much I use - six imports are needed:
- chart studio plotly: main plotly library
- plotly offline: use for offline-mode plotting
- plotly graph_objects: used to manipulate components of graphing objects
- plotly subplots: used to create a grid of plots
- plotly express: useful for some extra "quick and dirty" plotting
- plotly tools: duhh (I don't use often)

In [ ]:
import chart_studio.plotly as ply
import plotly.offline as plyoff
import plotly.graph_objects as go
import plotly.subplots as plysub
import plotly.express as px
import plotly.tools as plytool

## In offline mode, you must initialize plotly

In [ ]:
# to use plotly offline, need to initialize with a plot
plyoff.init_notebook_mode(connected=True)
init = go.Figure(data=[go.Scatter({'x':[1, 2], 'y':[42, 42], 'mode':'markers'})],
                 layout=go.Layout(title='Init', xaxis={'title':'x'}))
plyoff.iplot(init)

## Plot output
- when using jupyter notebooks, plots can be saved inline with the notebook, or as stand-alone html files
- in offline mode the former uses `plyoff.iplot()`, the latter uses `plyoff.plot()` 

### Inline
#### Pro
- easier to view code, data, analysis, and results - no tabbing
- facilitates keeping code & results together for collaboration & version contro

#### Con
- dramatically inflates notebook size, often passing the git limit of 2MB
- a notebook with many plots can be slow to load / refresh

### Stand-alone
#### Pro
- facilitates sharing only plots
- keeps notebooks concise and small
- allows keeping multiple versions of the same plot - just change the saved filename

#### Con
- requires extra work to ensure consistency between code & results
- saved html files consume more disk space than the plots do in a notebook, often passing the git limit of 2 MB

## Plotly Express
- "pre-packaged" plot functions for quick-and-dirty plotting
- fewer options for a plot
- mostly work by passing a dataframe and specifying column names
- I use plotly express only rarely

In [ ]:
# examples = scatter plot, histogram, box plot
px.scatter(data, x='X0', y='target', title='X0 vs. Target', color='Test').show()
px.histogram(data, x='resid', color='Test', title='Residuals Distribution').show()
px.box(data, y='target', x='a_string').show()

<a id='bild'></a>
## Building plotly figures
- a plot is built with a `go.Figure()` object, which holds `data` and `layout` elements
- these elements can be accessed using dict-style indexing, i.e. `fig['data']` or `fig['layout']`
- in fact, a plotly figure *could* be created just as a dictionary, though this would bypass some of plotly's integrity checks

### Data
- the data in a plot is specified as a list or tuple of plotly trace objects
- there a many types of trace graph objects:
    - <a href=#scat>Scatter & Scatter3d</a>
    - <a href=#bar>Bar</a>
    - <a href=#hist>Histogram</a>
    - <a href=#box>Box</a>
    - [Heatmap](https://plotly.com/python/heatmaps/)
    - [Pie](https://plotly.com/python/pie-charts/)
    - etc...
- the trace controls most of the ways that the data is plotted
- a single plot can contain multiple types of traces, i.e. a bar chart with a line plot
- more traces can be added to an existing figure with the `go.Figure.add_trace()` method
- any part of an existing trace can be modified after creation, i.e. `fig['data'][0]['x'] = ...`

### Layout
- the layout controls titles, axis labels, axis ticks, plot size, annotations, legend options, etc.
- the layout can be defined as a `go.Layout()` object
- the layout can also be defined / updated with the `update_layout()` method of the `go.Figure` class
- it can also be updated using dict-style indexing, i.e. `fig['layout']['xaxis']['title'] = ...`

In [ ]:
# see an example
print(init['data'])
print(init['data'][0]['y'])
print(init['layout'])
print(init['layout']['xaxis']['title'])

<a id='scat'></a>
## [Scatter Plots](https://plotly.com/python/line-and-scatter/)
- the `Scatter` class includes plain scatter plots, bubble plots, and line plots
- the `mode` attribute controls if the points are simply points (`mode='markers'`), lines (`mode='lines'`), text (`mode='text'`), or any combination thereof (`mode='markers+text'`)
- `Scatter3d` can generate 3 dimensional scatter and bubble plots
- for date-based plots, there are special options for displaying and interacting with [Time Series](https://plotly.com/python/time-series/)

In [ ]:
''' bubble plot with least squares fit line, markers sized by model error, colored by train / test status '''

# prepare the fit line
linX = np.linspace(data['target'].values.min(), data['yhat'].values.max(), 10).reshape(-1, 1)
fitlin = LinearRegression(n_jobs=-1)
fitlin.fit(X=data['target'].values.reshape(-1, 1), y=data['yhat'].values.reshape(-1, 1))
linY = fitlin.predict(X=linX)

# build the traces
trcs = [go.Scatter(x=data['target'], y=data['yhat'], mode='markers', name='data', text=['resid = %0.2f'%v for v in data['resid'].values],
                   hoverinfo='x+y+text', marker={'size':data['resid'].abs()*10, 'color':np.where(data['Test'], '#008000', '#FF0000')}), 
        go.Scatter(x=linX.squeeze(), y=linY.squeeze(), mode='lines', line={'color':'black', 'width':1}, name='fit line', showlegend=False)]

# set the layout
lout = go.Layout(title='$Y\\text{ vs. }\\hat{Y}$')
lout['xaxis1']['title'] = '$Y$'
lout['yaxis1']['title'] = '$\hat{Y}$'

# put the plot together
fig = go.Figure(data=trcs, layout=lout)

# show it
plyoff.iplot(fig)
if saveFigs:
    plyoff.plot(fig, filename='./scatter1.html', auto_open=True, include_mathjax='cdn')

### Of Note:
- LaTeX code in titles: Annotations can include html, markdown, and LaTeX; if saving a figure to html, need to "include_mathjax='cdn'" in the plyoff.plot() call if this advanced markup is used.
- sizing of bubbles: Bubble sizes are relative to figure size; I have always just set them through trial-and-error.
- legend: In this case, the legend is uninformative about colors; this is because the data was created as a single trace.
- marker & line colors: Colors can be defined with numbers, names ('red', 'green'), RGB codes, or hex codes.
- hover text: The text shown on hover is controlled by the `hoverinfo` attribute, set to any combination of 'x', 'y', or 'text'; if 'text' is included, it comes from the trace's `text` attribute.

In [ ]:
''' same as above, but with data separated to traces by train/test split & nicer hover text'''

# template
temp = '<i>Response</i>: %{x:0.2f}' + '<br><b>Prediction</b>: %{y:0.2f}' + '<br><b><i>Residual</i></b>: %{text}'

# build the traces
trcs = [go.Scatter(x=data.loc[data['Test']==0, 'target'], y=data.loc[data['Test']==0, 'yhat'], mode='markers',
                   marker={'size':data.loc[data['Test']==0, 'resid'].abs()*10, 'color':'red'}, name='train data',
                   text=['%0.2f'%v for v in data.loc[data['Test']==0, 'resid'].values], hovertemplate=temp),
        go.Scatter(x=data.loc[data['Test']==1, 'target'], y=data.loc[data['Test']==1, 'yhat'], mode='markers',
                   marker={'size':data.loc[data['Test']==1, 'resid'].abs()*10, 'color':'green'}, name='test data',
                   text=['%0.2f'%v for v in data.loc[data['Test']==0, 'resid'].values], hovertemplate=temp)]

# set the layout
lout = go.Layout(title='$Y\\text{ vs. }\\hat{Y}$')
lout['xaxis1']['title'] = '$Y$'
lout['yaxis1']['title'] = '$\hat{Y}$'

# put the plot together, then add in the fit line
fig = go.Figure(data=trcs, layout=lout)
fig.add_trace(go.Scatter(x=linX.squeeze(), y=linY.squeeze(), mode='lines', line={'color':'black', 'width':1}, name='fit line', showlegend=False))

# show it
plyoff.iplot(fig)
if saveFigs:
    plyoff.plot(fig, filename='./scatter2.html', auto_open=True, include_mathjax='cdn')

### Of note
- legend: The legend is more useful now, it can be used to toggle the visibility of individual traces; the legend entry can be hidden by trace (as done here) or for the entire plot (in the layout with the `showlegend` attribute).
- `add_trace()`: Though not necessary, note how the fit line was added after the figure was created; I could have also just appended the trace to the `trcs` list, then set it to `fig[data]=trcs`.
- hover text: For a fancier display of the hover text, the `hovertemplate` attribute can be specified, which can use html tags.

In [ ]:
''' 3d scatter plot '''

# build the traces
trcs = [go.Scatter3d(x=data.loc[data['Test']==i, 'X3'], y=data.loc[data['Test']==i, 'X2'], z=data.loc[data['Test']==i, 'target'], mode='markers',
                   marker={'size':data.loc[data['Test']==i, 'resid'].abs()*10, 'color':c}, name=n) for (i, (c, n)) in enumerate(zip(('red', 'green'), ('train', 'test')))]

# prepare the layout
lout = go.Layout(title='$X_3\\text{ vs. }X_2\\text{ vs. }Y$')

# put the plot together
fig = go.Figure(data=trcs, layout=lout)

# show it
plyoff.iplot(fig)
if saveFigs:
    plyoff.plot(fig, filename='./scatter3.html', auto_open=True, include_mathjax='cdn')

In [ ]:
''' time series plot '''

# group (filtered so labels show well without changing plot width)
gbd = data[data['a_date'].dt.year>=2016].groupby(by=['a_date'], as_index=False)['resid'].mean()
display(gbd.head())


# build the traces
trcs = [go.Scatter(x=gbd['a_date'], y=gbd['resid'], mode='markers+lines', marker={'color':'red', 'symbol':'triangle-down'},
                   line={'color':'blue'}, name='Residuals'),
        go.Scatter(x=gbd['a_date'], y=[0]*len(gbd['a_date']), mode='lines', line={'color':'black', 'width':1}, name='zero line')]

# set the layout
lout = go.Layout(title='Residuals over Time', showlegend=False, xaxis={'range':[gbd['a_date'].min(), gbd['a_date'].max()]})

# put the plot together
fig = go.Figure(data=trcs, layout=lout)
fig.update_xaxes(showgrid=True, ticklabelmode='period', dtick='M1', tickformat='%b\n%Y', rangeslider_visible=True,
                 rangeselector={'buttons':[{'count':1, 'label':'1M', 'step':'month', 'stepmode':'backward'},
                                          {'count':6, 'label':'6M', 'step':'month', 'stepmode':'backward'},
                                          {'count':1, 'label':'1Y', 'step':'year', 'stepmode':'backward'},
                                          {'step':'all'}]})

# show it
plyoff.iplot(fig)
if saveFigs:
    plyoff.plot(fig, filename='./scatter4.html', auto_open=True, include_mathjax='cdn')

### Of note:
- time series: This plot showcases several special time series axis date label options, a slider, and date filters.
- colors: It probably should go without being said, but the line & marker colors for a trace *don't* have to match.
- axis range: Plotly can determine the range for an axis automtically, or you can specify it in the `range` attribute.

<a href=#bild>Go To Building Plotly Figures</a>

<a id='bar'></a>
## [Bar Charts](https://plotly.com/python/bar-charts/)
- the `Bar` class can be used to make horizontal and vertical bar charts by setting the `orientation` ('h' or 'v')
- if a figure has a `Bar` (or `Histogram` or `Box`) object, the layout `barmode` can be set to 'group' or 'stack' to toggle grouped or stacked bar charts; there are other options as well I've not used for bar charts
- the order of the category axis (changes with orientation) can be controled in the layout with the `categoryorder` attribute of an exis; choices include specifying a list, 'category descending', 'total descending' (or the last two with 'ascending')

In [ ]:
''' horizontal grouped bar chart '''

# group
gbd = data.groupby(by=['a_string', 'Test'], as_index=False)['resid'].mean()
display(gbd)

# build the traces
trcs = [go.Bar(x=gbd.loc[gbd.Test==0, 'resid'], y=gbd.loc[gbd.Test==0, 'a_string'], text=['%0.2f'%v for v in gbd.loc[gbd.Test==0, 'resid'].values.tolist()],
               textposition='inside', orientation='h', name='Train', marker={'color':'red'}),
        go.Bar(x=gbd.loc[gbd.Test==1, 'resid'], y=gbd.loc[gbd.Test==1, 'a_string'], text=['%0.2f'%v for v in gbd.loc[gbd.Test==1, 'resid'].values.tolist()],
               textposition='outside', orientation='h', name='Test', marker={'color':'green'})]

# set the layout
lout = go.Layout(title='Mean Residual by Group', barmode='group', yaxis={'categoryorder':'category descending'})

# put the plot together
fig = go.Figure(data=trcs, layout=lout)

# show it
plyoff.iplot(fig)
if saveFigs:
    plyoff.plot(fig, filename='./bar1.html', auto_open=True, include_mathjax='cdn')

### Of note:
- category ordering: The y axis is the category axis, and ordered alphabetically.
- labels: Bar labels can be placed just inside or just outside the bar, controlled by the `textposition` attribute

In [ ]:
''' vertical stacked bar chart '''

# group
gbd = data.groupby(by=['a_string', 'Test'], as_index=False)['absresid'].mean()
display(gbd)

# build the traces
trcs = [go.Bar(x=gbd.loc[gbd.Test==0, 'a_string'], y=gbd.loc[gbd.Test==0, 'absresid'], text=['%0.2f'%v for v in gbd.loc[gbd.Test==0, 'absresid'].values.tolist()],
               textposition='auto', orientation='v', name='Train', marker={'color':'red'}),
        go.Bar(x=gbd.loc[gbd.Test==1, 'a_string'], y=gbd.loc[gbd.Test==1, 'absresid'], text=['%0.2f'%v for v in gbd.loc[gbd.Test==1, 'absresid'].values.tolist()],
               textposition='auto', orientation='v', name='Test', marker={'color':'green'})]

# build the traces
lout = go.Layout(title='Mean Absolute Residual by Group', barmode='stack', xaxis={'categoryorder':'total ascending'})

# put the plot together
fig = go.Figure(data=trcs, layout=lout)

# show it
plyoff.iplot(fig)
if saveFigs:
    plyoff.plot(fig, filename='./bar2.html', auto_open=True, include_mathjax='cdn')

### Of note:
- category ordering: Here, the categories are ordered by stack total, ascending.

In [ ]:
''' Pareto chart '''

# group
gbd = data.groupby(by=['a_string'], as_index=False)['resid'].mean()
gbd['resid'] = gbd['resid'].abs()
gbd.sort_values(by=['resid'], ascending=False, inplace=True)
gbd['residcum'] = gbd['resid'].cumsum()

display(gbd)

# build the traces
trcs = [go.Bar(x=gbd['a_string'].values.tolist(), y=gbd['resid'], text=['%0.2f'%v for v in gbd['resid'].values.tolist()], textposition='auto',
               name='Mean'),
        go.Scatter(x=gbd['a_string'], y=gbd['residcum'], mode='markers+lines+text', text=['%0.2f'%v for v in gbd['residcum'].values.tolist()],
                   textposition='top center', marker={'symbol':'star', 'color':'black'}, line={'color':'black', 'width':2},
                   name='Cumulative Mean')]

# set the layout
lout = go.Layout(title='Mean Residual by Group', barmode='group')

# put the plot together
fig = go.Figure(data=trcs, layout=lout)

# show it
plyoff.iplot(fig)
if saveFigs:
    plyoff.plot(fig, filename='./bar3.html', auto_open=True, include_mathjax='cdn')

### Of note:
- traces: This chart combined a `Bar` plot and `Scatter` plot to make the Pareto chart
- scatter plot: The cumulative line trace combined markers, lines, and text.
- category ordering: Categories were ordered by value descending by sorting data, rather than setting the `categoryorder` attribute.

<a href=#bild>Go To Building Plotly Figures</a>

<a id='hist'></a>
## [Histogram](https://plotly.com/python/histograms/)
- in the `Histogram` class, the `histnorm` attribute controls bar normalization:
    - 'percent' causes the bar heights to indicate the relative frequency
    - 'probability density' normalizes the bar areas so they all sum to 1
    - 'density' normalizes so bar areas sum to number of observations
- histograms can be stacked / overlaid or horizonal / vertical
- the `cumulative_enabled` attribute can generate a cumulative histogram

In [ ]:
''' grouped pdf with a few theoretical distribution curves '''

# create a few pdf overlays
mn = data['resid'].mean()
sd = data['resid'].std()
xs = np.linspace(-3, 3, 100)
gys = stt.norm.pdf(xs, loc=mn, scale=sd)
tys = stt.t.pdf(xs, df=3, loc=mn, scale=sd)
lys = stt.laplace.pdf(xs, loc=mn, scale=sd)

# build traces
trcs = [go.Histogram(x=data.loc[data.Test==0, 'resid'], histnorm='probability density', name='Train', nbinsx=20,
                     marker={'color':'red', 'opacity':0.75}),
        go.Histogram(x=data.loc[data.Test==1, 'resid'], histnorm='probability density', name='Test', nbinsx=20,
                     marker={'color':'green', 'opacity':0.75}),
        go.Scatter(x=xs, y=gys, mode='lines', line={'color':'black', 'dash':'solid'}, name='Fit Gaussian'),
        go.Scatter(x=xs, y=tys, mode='lines', line={'color':'blue', 'dash':'dash'}, name='Fit Student t'),
        go.Scatter(x=xs, y=lys, mode='lines', line={'color':'purple', 'dash':'dot'}, name='Fit Laplace')]

# set the layout
lout = go.Layout(title='Residuals Empirical Distribution', barmode='overlay')

# put the plot together
fig = go.Figure(data=trcs, layout=lout)

# show it
plyoff.iplot(fig)
if saveFigs:
    plyoff.plot(fig, filename='./histogram1.html', auto_open=True, include_mathjax='cdn')

### Of note:
- normalization: This histograms was normalized to a 20 bin-representation of the empirical probability density function, so it's morphology could be compared to the theoretical pdfs.
- line style: The theoretical distribution pdf curves were displayed with different line styles as well as colors.

In [ ]:
''' grouped horizontal histograms '''

# build the traces
trcs = [go.Histogram(y=data.loc[data['Test']==0, 'a_date'].dt.strftime('%b'), histnorm='percent', name='Train',
                     marker={'color':'red'}),
        go.Histogram(y=data.loc[data['Test']==1, 'a_date'].dt.strftime('%b'), histnorm='percent', name='Test',
                     marker={'color':'green'})]

# set the layout
lout = go.Layout(title='Data Count Distribution by Month', barmode='group',
                 yaxis={'categoryorder':'array', 'categoryarray':['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']})

# put the plot together
fig = go.Figure(data=trcs, layout=lout)

# show it
plyoff.iplot(fig)
if saveFigs:
    plyoff.plot(fig, filename='./histogram2.html', auto_open=True, include_mathjax='cdn')

### Of note:
- orientatin: The histogram is made horizontal simply by setting the data to `y` instead of `x`.
- category ordering: Setting the categories to integer months makes the ordering work well, but doesn't look nice; seting them to month names messes up the ordering; the solution is to hardcode the order by setting the `categoryorder` to 'array' and specifying the `categoryarray` attribute as an array-like.

<a href=#bild>Go To Building Plotly Figures</a>

<a id='box'></a>
## [Box Plots](https://plotly.com/python/box-plots/)
- the `Box` class includes several options for controlling how the boxes, whiskers, and data are defined & shown
- box plots can be oriented horizontally or vertically, and grouped or overlaid

In [ ]:
''' show options for points with horizontal boxes '''

# first make some big residuals
resd = data['resid'].sort_values().values
resd[:2] = resd[:2]*2
resd[-2:] = resd[-2:]*2

# build traces
trcs = [go.Box(x=resd, marker={'color':'orange'}, line={'color':'orange'}, name='All Points', jitter=0.3, pointpos=-1.8,
               boxpoints='all', boxmean=True),
        go.Box(x=resd, marker={'color':'green'}, line={'color':'green'}, name='Only Whiskers', boxpoints=False, boxmean=True),
        go.Box(x=resd, marker={'color':'blue', 'outliercolor':'red', 'line':{'outliercolor':'red'}}, line={'color':'blue'},
               boxmean=True, name='Outlier Suspects', boxpoints='suspectedoutliers'),
        go.Box(x=resd, marker={'color':'purple'}, line={'color':'purple'}, name='Whiskers & Outliers', boxpoints='outliers',
               boxmean=True)]

# set the layout
lout = go.Layout(title='Residuals')

# put the plot together
fig = go.Figure(data=trcs, layout=lout)

# show it
plyoff.iplot(fig)
if saveFigs:
    plyoff.plot(fig, filename='./box1.html', auto_open=True, include_mathjax='cdn')

### Of note:
- styles: This plot shows different options for styles of boxes vis-a-vis how the points are shown.
- suspected: I'm unsure how plotly defines a "suspected outlier"; clearly looks odd in this example.

In [ ]:
''' grouped horizontal box plots '''

# build the traces
trcs = [go.Box(x=data.loc[data['Test']==0, 'a_date'].dt.strftime('%b'), y=data.loc[data['Test']==0, 'resid'],
               boxpoints='outliers', boxmean='sd', marker={'color':'red'}, line={'color':'red'}, name='Train'),
        go.Box(x=data.loc[data['Test']==1, 'a_date'].dt.strftime('%b'), y=data.loc[data['Test']==1, 'resid'],
               boxpoints='outliers', boxmean='sd', marker={'color':'green'}, line={'color':'green'}, name='Test')]

# set the layout
lout = go.Layout(title='Train / Test Residuls by Month', boxmode='group',
                 xaxis={'categoryorder':'array', 'categoryarray':['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']})

# put the plot together
fig = go.Figure(data=trcs, layout=lout)

# show it
plyoff.iplot(fig)
if saveFigs:
    plyoff.plot(fig, filename='./box2.html', auto_open=True, include_mathjax='cdn')

### Of Note:
- mode: The box plots are grouped together, or not, by specifying the `boxmode` attribute.
- mean display: Setting `boxmean` to 'sd' causes the mean and $\mu \pm 1 \sigma$ to show in dashed lines; setting it to True just shows the mean; setting it to None hides mean information entirely.
- orientation: The orientation is defined by the data being set to either `x` or `y`, as with `Histogram`.

<a href=#bild>Go To Building Plotly Figures</a>

## [Subplots](https://plotly.com/python/subplots/)
- a subplots figure - multiple non-overlapping plots in the same figure - can be generated using `plotly.subplots`
- a subplots figure is generated by specifying the shape initially, then adding traces to the plots using **1-based** (row, column) specifications, with the `add_trace` method
- sub-plots within the figure can be blank, and some can be merged
- sub-plots are by default distributed evenly, but can be resized
- in addition to an overall title for the entire figure, each sub-plot can have it's own title
- traces in the sub-plots can be connected to each other for toggling visibility

In [ ]:
''' 4x4 subplot figure with different sizes'''

# group
gbd = data.groupby(by=['a_date', 'Test'], as_index=False)['resid'].mean()

# prep the figure
fig = plysub.make_subplots(rows=2, cols=2, print_grid=False,
                           subplot_titles=['Responses vs Predictions', 'Residuals Distribution','Mean Residuals by Date', 'Responses vs Residuals'])

# build the traces
fig.add_trace(go.Scatter(x=data.loc[data['Test']==0, 'target'], y=data.loc[data['Test']==0, 'yhat'], name='Train R vs P',
                         mode='markers', marker={'color':'red'}, legendgroup='Train'), 1, 1)
fig.add_trace(go.Scatter(x=data.loc[data['Test']==1, 'target'], y=data.loc[data['Test']==1, 'yhat'], name='Test R vs P',
                         mode='markers', marker={'color':'green'}, legendgroup='Test'), 1, 1)

fig.add_trace(go.Histogram(x=data['resid'], marker={'color':'red'}, name='Residuals', nbinsx=20, histnorm='probability density'), 1, 2)
fig.add_trace(go.Scatter(x=xs, y=gys, mode='lines', line={'color':'black', 'dash':'solid'}, name='Fit Gaussian', showlegend=False), 1, 2)

fig.add_trace(go.Scatter(x=gbd.loc[gbd['Test']==0, 'a_date'], y=gbd.loc[gbd['Test']==0, 'resid'], mode='markers+lines',
                         marker={'color':'red', 'symbol':'triangle-down'}, line={'color':'red'}, name='Train D vs R',
                         legendgroup='Train'), 2, 1)
fig.add_trace(go.Scatter(x=gbd.loc[gbd['Test']==1, 'a_date'], y=gbd.loc[gbd['Test']==1, 'resid'], mode='markers+lines',
                         marker={'color':'green', 'symbol':'triangle-up'}, line={'color':'green'}, name='Test D vs R',
                         legendgroup='Test'), 2, 1)

fig.add_trace(go.Scatter(x=data.loc[data['Test']==0, 'target'], y=data.loc[data['Test']==0, 'resid'], name='Train R vs R',
                         mode='markers', marker={'color':'red'}, legendgroup='Train'), 2, 2)
fig.add_trace(go.Scatter(x=data.loc[data['Test']==1, 'target'], y=data.loc[data['Test']==1, 'resid'], name='Test R vs R',
                         mode='markers', marker={'color':'green'}, legendgroup='Test'), 2, 2)

# set the layout
fig.update_layout(title='Results Plots', height=1500, width=1500)
fig['layout']['xaxis1'].update(title='Response', domain=[0.0, 0.35])
fig['layout']['yaxis1'].update(title='Prediction')
fig['layout']['xaxis2'].update(title='Residual', domain=[0.4, 1.0])
fig['layout']['yaxis2'].update(range=[0.0, 0.5])
fig['layout']['xaxis3'].update(showgrid=True, ticklabelmode='period', range=[gbd['a_date'].min(), gbd['a_date'].max()],
                               dtick='M2', tickformat='%b\n%Y', rangeslider_visible=False,
                               rangeselector={'buttons':[{'count':1, 'label':'1M', 'step':'month', 'stepmode':'backward'},
                                          {'count':6, 'label':'6M', 'step':'month', 'stepmode':'backward'},
                                          {'count':1, 'label':'1Y', 'step':'year', 'stepmode':'backward'},
                                          {'step':'all'}]}, title='Date', domain=[0.0, 0.6])
fig['layout']['yaxis3'].update(title='Residual')
fig['layout']['xaxis4'].update(title='Response', domain=[0.65, 1.0])

# show it
plyoff.iplot(fig)
if saveFigs:
    plyoff.plot(fig, filename='./subplot1.html', auto_open=True, include_mathjax='cdn')

### Of note:
- sizes: The axis `domain` attribute, ranging from 0.0 to 1.0, specifies how much of the total width (x axis) or height (y axis) a sub-plot can span.
- legend grouping: Setting the `legendgroup` attribute to the same value in multiple traces allows simultaneous toggling of visibility for those traces.
- setting axis attributes: I've found that it's generally best to set axes attributes after all traces have been added, so axes all exist with expected counts.

In [ ]:
''' subplot figure with uneven sizes & merging '''

# prep the figure
fig = plysub.make_subplots(rows=2, cols=2, specs=[[{'colspan':2}, None], [{}, {}]], print_grid=True, subplot_titles=['PDF: All', 'PDF: Train', 'PDF: Test'])

# build the traces
fig.add_trace(go.Scatter(x=xs, y=gys, mode='lines', line={'color':'black', 'dash':'solid'}, name='Fit Gaussian', legendgroup='Fit Gaussian'), 1, 1)
fig.add_trace(go.Histogram(x=data['resid'], marker={'color':'blue'}, name='All', nbinsx=20, histnorm='probability density'), 1, 1)

fig.add_trace(go.Histogram(x=data.loc[data['Test']==0, 'resid'], marker={'color':'red'}, name='Train', nbinsx=20, histnorm='probability density'), 2, 1)
fig.add_trace(go.Scatter(x=xs, y=gys, mode='lines', line={'color':'black', 'dash':'solid'}, name='Fit Gaussian', legendgroup='Fit Gaussian', showlegend=False), 2, 1)

fig.add_trace(go.Histogram(x=data.loc[data['Test']==1, 'resid'], marker={'color':'green'}, name='Test', nbinsx=20, histnorm='probability density'), 2, 2)
fig.add_trace(go.Scatter(x=xs, y=gys, mode='lines', line={'color':'black', 'dash':'solid'}, name='Fit Gaussian', legendgroup='Fit Gaussian', showlegend=False), 2, 2)

# set the layout
fig['layout'].update(title='Residuals Empirical Distributions', height=1000, width=1000)
fig['layout']['yaxis1'].update(domain=[0.3, 1.0], range=[0.0, 0.7])
fig['layout']['yaxis2'].update(domain=[0.0, 0.25], range=[0.0, 0.7])
fig['layout']['yaxis3'].update(domain=[0.0, 0.25], range=[0.0, 0.7])
fig['layout']['annotations'][1].update(y=0.25)
fig['layout']['annotations'][2].update(y=0.25)

# show it
plyoff.iplot(fig)
if saveFigs:
    plyoff.plot(fig, filename='./subplot2.html', auto_open=True, include_mathjax='cdn')

### Of note:
- legend grouping: There were 3 traces in the "fit Gaussian" legend group with the same name - `showlegend` was set to false on 2 of them so only one shows and controls the toggling.
- structure: The `specs` attribute was used to have the top 2 plot loctions merged.
- title placement: The bottom two plots were shrunk using the y-axis `domain` attribute, but that messed up placement of their titles; I solves this by updating the `y` attribute of the `annotations` list of the figure layout to correct the bottom plots' title placements.

In [ ]:
# let's inspect the annotations
print(fig['layout']['annotations'])

<a href=#bild>Building Plotly Figures</a>

## Colors
For named colors (rather than RGB or Hex codes), we don't need to just stick with simple colors as I've done here. For reference, here's a complete listing of plotly named colors.

<font color="aliceblue">aliceblue</font>
-<font color="antiquewhite">antiquewhite</font>
-<font color="aqua">aqua</font>
-<font color="aquamarine">aquamarine</font>
-<font color="azure">azure</font>
-<font color="beige">beige</font>
-<font color="bisque">bisque</font>
-<font color="black">black</font>
-<font color="blanchedalmond">blanchedalmond</font>
-<font color="blue">blue</font>
-<font color="blueviolet">blueviolet</font>
-<font color="brown">brown</font>
-<font color="burlywood">burlywood</font>
-<font color="cadetblue">cadetblue</font>
-<font color="chartreuse">chartreuse</font>
-<font color="chocolate">chocolate</font>
-<font color="coral">coral</font>
-<font color="cornflowerblue">cornflowerblue</font>
-<font color="cornsilk">cornsilk</font>
-<font color="crimson">crimson</font>
-<font color="cyan">cyan</font>
-<font color="darkblue">darkblue</font>
-<font color="darkcyan">darkcyan</font>
-<font color="darkgoldenrod">darkgoldenrod</font>
-<font color="darkgray">darkgray</font>
-<font color="darkgrey">darkgrey</font>
-<font color="darkgreen">darkgreen</font>
-<font color="darkkhaki">darkkhaki</font>
-<font color="darkmagenta">darkmagenta</font>
-<font color="darkolivegreen">darkolivegreen</font>
-<font color="darkorange">darkorange</font>
-<font color="darkorchid">darkorchid</font>
-<font color="darkred">darkred</font>
-<font color="darksalmon">darksalmon</font>
-<font color="darkseagreen">darkseagreen</font>
-<font color="darkslateblue">darkslateblue</font>
-<font color="darkslategray">darkslategray</font>
-<font color="darkslategrey">darkslategrey</font>
-<font color="darkturquoise">darkturquoise</font>
-<font color="darkviolet">darkviolet</font>
-<font color="deeppink">deeppink</font>
-<font color="deepskyblue">deepskyblue</font>
-<font color="dimgray">dimgray</font>
-<font color="dimgrey">dimgrey</font>
-<font color="dodgerblue">dodgerblue</font>
-<font color="firebrick">firebrick</font>
-<font color="floralwhite">floralwhite</font>
-<font color="forestgreen">forestgreen</font>
-<font color="fuchsia">fuchsia</font>
-<font color="gainsboro">gainsboro</font>
-<font color="ghostwhite">ghostwhite</font>
-<font color="gold">gold</font>
-<font color="goldenrod">goldenrod</font>
-<font color="gray">gray</font>
-<font color="grey">grey</font>
-<font color="green">green</font>
-<font color="greenyellow">greenyellow</font>
-<font color="honeydew">honeydew</font>
-<font color="hotpink">hotpink</font>
-<font color="indianred">indianred</font>
-<font color="indigo">indigo</font>
-<font color="ivory">ivory</font>
-<font color="khaki">khaki</font>
-<font color="lavender">lavender</font>
-<font color="lavenderblush">lavenderblush</font>
-<font color="lawngreen">lawngreen</font>
-<font color="lemonchiffon">lemonchiffon</font>
-<font color="lightblue">lightblue</font>
-<font color="lightcoral">lightcoral</font>
-<font color="lightcyan">lightcyan</font>
-<font color="lightgoldenrodyellow">lightgoldenrodyellow</font>
-<font color="lightgray">lightgray</font>
-<font color="lightgrey">lightgrey</font>
-<font color="lightgreen">lightgreen</font>
-<font color="lightpink">lightpink</font>
-<font color="lightsalmon">lightsalmon</font>
-<font color="lightseagreen">lightseagreen</font>
-<font color="lightskyblue">lightskyblue</font>
-<font color="lightslategray">lightslategray</font>
-<font color="lightslategrey">lightslategrey</font>
-<font color="lightsteelblue">lightsteelblue</font>
-<font color="lightyellow">lightyellow</font>
-<font color="lime">lime</font>
-<font color="limegreen">limegreen</font>
-<font color="linen">linen</font>
-<font color="magenta">magenta</font>
-<font color="maroon">maroon</font>
-<font color="mediumaquamarine">mediumaquamarine</font>
-<font color="mediumblue">mediumblue</font>
-<font color="mediumorchid">mediumorchid</font>
-<font color="mediumpurple">mediumpurple</font>
-<font color="mediumseagreen">mediumseagreen</font>
-<font color="mediumslateblue">mediumslateblue</font>
-<font color="mediumspringgreen">mediumspringgreen</font>
-<font color="mediumturquoise">mediumturquoise</font>
-<font color="mediumvioletred">mediumvioletred</font>
-<font color="midnightblue">midnightblue</font>
-<font color="mintcream">mintcream</font>
-<font color="mistyrose">mistyrose</font>
-<font color="moccasin">moccasin</font>
-<font color="navajowhite">navajowhite</font>
-<font color="navy">navy</font>
-<font color="oldlace">oldlace</font>
-<font color="olive">olive</font>
-<font color="olivedrab">olivedrab</font>
-<font color="orange">orange</font>
-<font color="orangered">orangered</font>
-<font color="orchid">orchid</font>
-<font color="palegoldenrod">palegoldenrod</font>
-<font color="palegreen">palegreen</font>
-<font color="paleturquoise">paleturquoise</font>
-<font color="palevioletred">palevioletred</font>
-<font color="papayawhip">papayawhip</font>
-<font color="peachpuff">peachpuff</font>
-<font color="peru">peru</font>
-<font color="pink">pink</font>
-<font color="plum">plum</font>
-<font color="powderblue">powderblue</font>
-<font color="purple">purple</font>
-<font color="red">red</font>
-<font color="rosybrown">rosybrown</font>
-<font color="royalblue">royalblue</font>
-<font color="saddlebrown">saddlebrown</font>
-<font color="salmon">salmon</font>
-<font color="sandybrown">sandybrown</font>
-<font color="seagreen">seagreen</font>
-<font color="seashell">seashell</font>
-<font color="sienna">sienna</font>
-<font color="silver">silver</font>
-<font color="skyblue">skyblue</font>
-<font color="slateblue">slateblue</font>
-<font color="slategray">slategray</font>
-<font color="slategrey">slategrey</font>
-<font color="snow">snow</font>
-<font color="springgreen">springgreen</font>
-<font color="steelblue">steelblue</font>
-<font color="tan">tan</font>
-<font color="teal">teal</font>
-<font color="thistle">thistle</font>
-<font color="tomato">tomato</font>
-<font color="turquoise">turquoise</font>
-<font color="violet">violet</font>
-<font color="wheat">wheat</font>
-<font color="white">white</font>
-<font color="whitesmoke">whitesmoke</font>
-<font color="yellow">yellow</font>
-<font color="yellowgreen">yellowgreen</font>

Some useful plotly color references can be found [here](https://plotly.com/python/discrete-color/) and [here](https://plotly.com/python/builtin-colorscales/).

In [ ]:
''' plotly colors reference '''
# colors
colors = ['aliceblue','antiquewhite','aqua','aquamarine','azure','beige','bisque','black','blanchedalmond','blue','blueviolet','brown','burlywood',
          'cadetblue','chartreuse','chocolate','coral','cornflowerblue','cornsilk','crimson','cyan','darkblue','darkcyan','darkgoldenrod','darkgray',
          'darkgrey','darkgreen','darkkhaki','darkmagenta','darkolivegreen','darkorange','darkorchid','darkred','darksalmon','darkseagreen','darkslateblue',
          'darkslategray','darkslategrey','darkturquoise','darkviolet','deeppink','deepskyblue','dimgray','dimgrey','dodgerblue','firebrick','floralwhite',
          'forestgreen','fuchsia','gainsboro','ghostwhite','gold','goldenrod','gray','grey','green','greenyellow','honeydew','hotpink','indianred','indigo',
          'ivory','khaki','lavender','lavenderblush','lawngreen','lemonchiffon','lightblue','lightcoral','lightcyan','lightgoldenrodyellow','lightgray',
          'lightgrey','lightgreen','lightpink','lightsalmon','lightseagreen','lightskyblue','lightslategray','lightslategrey','lightsteelblue','lightyellow',
          'lime','limegreen','linen','magenta','maroon','mediumaquamarine','mediumblue','mediumorchid','mediumpurple','mediumseagreen','mediumslateblue',
          'mediumspringgreen','mediumturquoise','mediumvioletred','midnightblue','mintcream','mistyrose','moccasin','navajowhite','navy','oldlace','olive',
          'olivedrab','orange','orangered','orchid','palegoldenrod','palegreen','paleturquoise','palevioletred','papayawhip','peachpuff','peru','pink','plum',
          'powderblue','purple','red','rosybrown','royalblue','saddlebrown','salmon','sandybrown','seagreen','seashell','sienna','silver','skyblue','slateblue',
          'slategray','slategrey','snow','springgreen','steelblue','tan','teal','thistle','tomato','turquoise','violet','wheat','white','whitesmoke','yellow','yellowgreen']
lenCol = len(colors)

# build traces
trcs = [None]*lenCol
for i, col in enumerate(colors):
    trcs[i] = go.Bar(x=[i+5], y=[42], marker={'color':col}, name=col, legendgroup=col)

# setup layouts
loutL = go.Layout(title='Light Background (default)', width=1500, height=500, xaxis={'range':[0, lenCol+5]})
loutD = go.Layout(title='Black Background', plot_bgcolor='black', width=1500, height=500, xaxis={'range':[0, lenCol+5]})

# put the plots together
figL = go.Figure(data=trcs, layout=loutL)
figD = go.Figure(data=trcs, layout=loutD)

# show them
plyoff.iplot(figL)
plyoff.iplot(figD)
if saveFigs:
    plyoff.plot(figL, filename='./colors_light.html', auto_open=True, include_mathjax='cdn')
    plyoff.plot(figD, filename='./colors_dark.html', auto_open=True, include_mathjax='cdn')

<a href=#bild>Go To Building Plotly Figures</a>

<a id='quiz'></a>
## Practice

In [ ]:
''' subplot: build a set of time series plots showing X0-X4 min, mean, and max by a_date, with a sub-plot for each feature '''


### My solution (font is white - enter cell to access)
<font color='white'>
# prep the data
cols = data.columns.values.tolist()[:5]
gbd = data.groupby(by=['a_date'])[cols].agg([min, np.mean, max]).reset_index()
gbd.columns = ['_'.join(col) for col in gbd.columns.values]
display(gbd.head())

# prep the figure
(numRows, numCols) = (3, 2)
fig = plysub.make_subplots(rows=numRows, cols=numCols, subplot_titles=['$%s_%s$'%(c[0], c[1]) for c in cols])

# build the traces
for i, col in enumerate(cols):
    # row & column
    frow = math.floor(i/numCols) + 1
    fcol = i-(frow - 1)*numCols + 1
    # min
    fig.add_trace(go.Scatter(x=gbd['a_date_'], y=gbd[col+'_min'], mode='markers+lines', marker={'color':'red', 'symbol':'triangle-down'},
                             line={'color':'red'}, name='min', legendgroup='min', showlegend=(i==0)), frow, fcol)
    # mean
    fig.add_trace(go.Scatter(x=gbd['a_date_'], y=gbd[col+'_mean'], mode='markers+lines', marker={'color':'blue', 'symbol':'star'},
                             line={'color':'blue'}, name='mean', legendgroup='mean', showlegend=(i==0)), frow, fcol)
    # max
    fig.add_trace(go.Scatter(x=gbd['a_date_'], y=gbd[col+'_max'], mode='markers+lines', marker={'color':'green', 'symbol':'triangle-up'},
                             line={'color':'green'}, name='max', legendgroup='max', showlegend=(i==0)), frow, fcol)

# set up the layout
fig.update_layout(title='Feature Time Plots', width=1500, height=1500)
fig['layout']['xaxis1'].update(ticklabelmode='period', range=[gbd['a_date_'].min(), gbd['a_date_'].max()], dtick='M2', tickformat='%b\n%Y')
fig['layout']['xaxis2'].update(ticklabelmode='period', range=[gbd['a_date_'].min(), gbd['a_date_'].max()], dtick='M2', tickformat='%b\n%Y')
fig['layout']['xaxis3'].update(ticklabelmode='period', range=[gbd['a_date_'].min(), gbd['a_date_'].max()], dtick='M2', tickformat='%b\n%Y')
fig['layout']['xaxis4'].update(ticklabelmode='period', range=[gbd['a_date_'].min(), gbd['a_date_'].max()], dtick='M2', tickformat='%b\n%Y')
fig['layout']['xaxis5'].update(ticklabelmode='period', range=[gbd['a_date_'].min(), gbd['a_date_'].max()], dtick='M2', tickformat='%b\n%Y')

# show it
plyoff.iplot(fig)
if saveFigs:
    plyoff.plot(fig, filename='./practice1.html', auto_open=True, include_mathjax='cdn')
</font>

In [ ]:
''' subplot: build a set of scatter plots of X0-X4 vs target; one sub-plot per feature '''


### My solution (font is white - enter cell to access)
<font color='white'>
# prep the figure
cols = data.columns.values.tolist()[:5]
numRows, numCols = (2, 3)
fig = plysub.make_subplots(rows=numRows, cols=numCols, subplot_titles=['$%s_%s\\text{ vs. }Y$'%(c[0], c[1]) for c in cols])

# build the traces
for i, col in enumerate(cols):
    # row & column
    frow = math.floor(i/numCols) + 1
    fcol = i-(frow - 1)*numCols + 1
    # trace
    fig.add_trace(go.Scatter(x=data[col], y=data['target'], mode='markers', marker={'color':'blue'}, name=col), frow, fcol)
    # layout
    fig['layout']['xaxis%d'%(i+1)].update(title='$%s_%s$'%(col[0], col[1]))

# set up the layout
fig.update_layout(title='Feature vs. Target Plots', width=1500, height=1500, showlegend=False)
fig['layout']['yaxis1'].update(title='Target')
fig['layout']['yaxis2'].update(title='Target')
fig['layout']['yaxis3'].update(title='Target')
fig['layout']['yaxis4'].update(title='Target')
fig['layout']['yaxis5'].update(title='Target')

# show it
plyoff.iplot(fig)
if saveFigs:
    plyoff.plot(fig, filename='./practice2.html', auto_open=True, include_mathjax='cdn')
</font>

In [ ]:
''' subplot: build a prob. density histogram of target, box plot of yhat, and stacked bar chart of test / train counts per months '''


### My solution (font is white - enter cell to access)
<font color='white'>
# prep the data
gbd = data.copy()
gbd['a_month'] = gbd['a_date'].dt.strftime('%b')
gbd['cnt'] = 1
gbd = gbd.groupby(by=['a_month', 'Test'], as_index=False)['cnt'].sum()
display(gbd.head())

# prep the figure
fig = plysub.make_subplots(rows=1, cols=3, subplot_titles=['Target PDF', 'Predictions Box Plot', 'Train/Test Counts by Month'])

# build the traces
fig.add_trace(go.Histogram(x=data['target'], histnorm='probability density', nbinsx=20, marker={'color':'purple'},
                           name='Target', showlegend=False), 1, 1)
fig.add_trace(go.Box(y=data['yhat'], boxpoints='outliers', boxmean='sd', marker={'color':'purple'}, name='Predictions',
                     showlegend=False), 1, 2)
fig.add_trace(go.Bar(x=gbd.loc[gbd.Test==0, 'a_month'], y=gbd.loc[gbd.Test==0, 'cnt'], orientation='v', name='Train',
                     marker={'color':'red'}, text=['%d'%v for v in gbd.loc[gbd.Test==0, 'cnt'].values.tolist()],  textposition='auto'), 1, 3)
fig.add_trace(go.Bar(x=gbd.loc[gbd.Test==1, 'a_month'], y=gbd.loc[gbd.Test==1, 'cnt'], orientation='v', name='Test',
                     marker={'color':'green'}, text=['%d'%v for v in gbd.loc[gbd.Test==1, 'cnt'].values.tolist()], textposition='auto'), 1, 3)


# setup the layout
fig.update_layout(title='Some Plots', width=1800, height=800, barmode='stack')
fig['layout']['xaxis1'].update(title='Target')
fig['layout']['xaxis3'].update(categoryorder='array', categoryarray=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

# show it
plyoff.iplot(fig)
if saveFigs:
    plyoff.plot(fig, filename='./practice3.html', auto_open=True, include_mathjax='cdn')
</font>

<a href=#titl>Go to top</a>